In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('train.csv')
# df = pd.read_csv('test.csv')

df.head(5)

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
2,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1
3,z13lfzdo5vmdi1cm123te5uz2mqig1brz04,ferleck ferles,2013-11-27T21:39:24,Subscribe to my channel ﻿,1
4,z12avveb4xqiirsix04chxviiljryduwxg0,BeBe Burkey,2013-11-28T16:30:13,and u should.d check my channel and tell me wh...,1


In [3]:
''' 
Here there is no direct connection between the comment id and the class label. becuase these comment ids are randomly generated.
which has no direct impact on class lable.
Here class lable is only dependet on the content of the comment.

SO, I can use the NLP for this purpose.
Here I gonna use pretrained NLP model fot this porpose.
And by using this pretrained model, I can get the some parameteres which are very useful for to predict the class label.

'''

' \nHere there is no direct connection between the comment id and the class label. becuase these comment ids are randomly generated.\nwhich has no direct impact on class lable.\nHere class lable is only dependet on the content of the comment.\n\nSO, I can use the NLP for this purpose.\nHere I gonna use pretrained NLP model fot this porpose.\nAnd by using this pretrained model, I can get the some parameteres which are very useful for to predict the class label.\n\n'

In [4]:
df.drop(['AUTHOR', 'COMMENT_ID', 'DATE'], axis=1, inplace=True)

In [5]:
len(df)

1157

In [6]:
labels=df.CLASS
labels.head()

0    1
1    1
2    1
3    1
4    1
Name: CLASS, dtype: int64

In [7]:
labels.value_counts()

1    586
0    571
Name: CLASS, dtype: int64

In [8]:
x = df.drop(['CLASS'], axis=1)
y = df['CLASS']

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, classification_report

In [10]:
x_train,x_test,y_train,y_test=train_test_split(df['CONTENT'],y, test_size=0.2, random_state=7)

In [11]:
#DataFlair - Initialize a TfidfVectorizer
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)


#DataFlair - Fit and transform train set, transform test set
tfidf_train=tfidf_vectorizer.fit_transform(x_train) 
tfidf_test=tfidf_vectorizer.transform(x_test)

In [12]:
#DataFlair - Initialize a PassiveAggressiveClassifier

pac=PassiveAggressiveClassifier(max_iter=50)

pac.fit(tfidf_train,y_train)

#DataFlair - Predict on the test set and calculate accuracy
y_pred=pac.predict(tfidf_test)

score=accuracy_score(y_test,y_pred)
roc = roc_auc_score(y_test, y_pred)

print(f'Accuracy: {round(score*100,2)}%')
print(f'Accuracy: {round(roc*100,2)}%')

print(classification_report(y_test,y_pred))

Accuracy: 89.22%
Accuracy: 89.16%
              precision    recall  f1-score   support

           0       0.91      0.87      0.89       113
           1       0.88      0.92      0.90       119

    accuracy                           0.89       232
   macro avg       0.89      0.89      0.89       232
weighted avg       0.89      0.89      0.89       232



In [13]:
#DataFlair - Build confusion matrix
confusion_matrix(y_test,y_pred, labels=[0,1])

array([[ 98,  15],
       [ 10, 109]], dtype=int64)

### Multinomial Naive Bayes

In [14]:
from sklearn.naive_bayes import MultinomialNB

In [15]:
clf = MultinomialNB()
clf.fit(tfidf_train, y_train)
y_prediction = clf.predict(tfidf_test)

In [16]:
score=accuracy_score(y_test,y_prediction)
roc = roc_auc_score(y_test, y_prediction)

print(f'Accuracy: {round(score*100,2)}%')
print(f'roc: {round(roc*100,2)}%')

Accuracy: 90.95%
roc: 90.82%


In [17]:
test_df = pd.read_csv('test.csv')
test_tfidf=tfidf_vectorizer.transform(test_df['CONTENT'])
y_pred=clf.predict(test_tfidf)

In [18]:
submisstion = pd.DataFrame()
submisstion['ID'] = test_df['ID']
submisstion['CLASS'] = y_pred
submisstion.to_csv('ans1.csv', index=False)

### nueral netowork model

In [20]:
from keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [49]:
input_dim = tfidf_train.shape[1]  # Number of features

# model = Sequential()
# model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
# model.add(layers.Dense(1, activation='sigmoid'))

ann_model = Sequential(layers=[
    Dense(units=1024,activation='relu',input_shape=[input_dim]),
    Dense(units=512,activation='tanh'),
    Dense(1,activation='sigmoid')]
)
ann_model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

In [50]:
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [51]:
ann_model.fit(tfidf_train.toarray(), y_train, epochs=10, batch_size=32)

Epoch 1/10
29/29 [==============================] - 1s 25ms/step - loss: 0.4507 - accuracy: 0.8086
Epoch 2/10
29/29 [==============================] - 1s 25ms/step - loss: 0.0572 - accuracy: 0.9805
Epoch 3/10
29/29 [==============================] - 1s 29ms/step - loss: 0.0070 - accuracy: 0.9989
Epoch 4/10
29/29 [==============================] - 1s 25ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 5/10
29/29 [==============================] - 1s 25ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 6/10
29/29 [==============================] - 1s 25ms/step - loss: 6.8208e-04 - accuracy: 1.0000
Epoch 7/10
29/29 [==============================] - 1s 25ms/step - loss: 5.0034e-04 - accuracy: 1.0000
Epoch 8/10
29/29 [==============================] - 1s 25ms/step - loss: 4.2584e-04 - accuracy: 1.0000
Epoch 9/10
29/29 [==============================] - 1s 25ms/step - loss: 3.3310e-04 - accuracy: 1.0000
Epoch 10/10
29/29 [==============================] - 1s 26ms/step - loss: 3.0852e-04 - accura

In [52]:
y_pr = ann_model.predict(tfidf_test.toarray())

In [53]:
f = []
for x in y_pr:
    if x >= 0.5:
        f.append(1)
    else:
        f.append(0)

In [55]:
ann_model.evaluate(tfidf_test.toarray(), y_test)

8/8 [==============================] - 0s 5ms/step - loss: 0.3446 - accuracy: 0.9095


[0.34459802508354187, 0.9094827771186829]

In [ ]:
submisstion = pd.DataFrame()
submisstion['ID'] = test_df['ID']
submisstion['CLASS'] = f
submisstion.to_csv('ans1.csv', index=False)

In [56]:
print(ann_model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 1024)              2622464   
                                                                 
 dense_16 (Dense)            (None, 512)               524800    
                                                                 
 dense_17 (Dense)            (None, 1)                 513       
                                                                 
Total params: 3,147,777
Trainable params: 3,147,777
Non-trainable params: 0
_________________________________________________________________
None


### Modified ann (colab)

In [16]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input,GlobalMaxPool1D,Dropout

import nltk
import re
from nltk.corpus import stopwords #corpus is collection of text
from nltk.stem.porter import PorterStemmer

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score

In [13]:
vo_size=500
messages=x.copy()
messages.head()

In [ ]:
#dataset Preprocessing
ps =PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print("Status: %s / %s" %(i, len(messages)), end="\r")
    review = re.sub('[^a-zA-Z]', ' ',messages['CONTENT'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [114]:
model = Sequential()
model.add(Embedding(500,int(100),input_length=input_dim))
model.add(LSTM(100, return_sequences=True,name='lstm_layer'))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.1))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [115]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [38]:
# model.fit(X_train, y_train, epochs=10)

### Pre-tranined model

In [ ]:
from simpletransformers.classification import ClassificationModel


# Create a TransformerModel
model = ClassificationModel('roberta', 'roberta-base')

# Train the model
model.train_model(df)

# Evaluate the model
# result, model_outputs, wrong_predictions = model.eval_model(eval_df)